In [2]:
# Import all necessary libraries

import transformers
import torch
import datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AutoTokenizer, AutoModelForTokenClassification, DataCollatorForTokenClassification
from datasets import load_dataset, load_metric, DatasetDict
from transformers import TrainingArguments, Trainer

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
# Activate GPU for training 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [4]:
# We needed to install the evaluation metric for this task

#!pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=fd2190b04c40cb5dd98c9f3dae0cb0c5bd1a1e06c6e8df7061726745510b600b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [6]:
# Import the dataset directly from HuggingFace

multinerd = load_dataset("Babelscape/multinerd")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/Babelscape--multinerd-d3bf0284fd817c7b/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
# Originally, two segments - train and test. Each segment contains tokens, ner_tags and information about language
multinerd

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 1339200
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 167993
    })
})

In [8]:
# Checking the dimensions of data

multinerd.shape

{'train': (1339200, 3), 'test': (167993, 3)}

In [9]:
# Filter out the non-English examples

multinerd = multinerd.filter(lambda sample: sample['lang'] == 'en')

  0%|          | 0/1340 [00:00<?, ?ba/s]

  0%|          | 0/168 [00:00<?, ?ba/s]

In [10]:
# Manually create a val set from training set, while maintaining train and test in their original form

train_dataset = multinerd['train']
test_dataset = multinerd['test']

split_ratio = 0.8
num_train_samples = int(len(train_dataset) * split_ratio)

train_set = train_dataset.select(list(range(num_train_samples)))
val_set = train_dataset.select(list(range(num_train_samples, len(train_dataset))))

# Update the DatasetDict with the sets
multinerd = DatasetDict({
    'train': train_set,
    'valid': val_set,
    'test': test_dataset
})

In [11]:
multinerd

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 105024
    })
    valid: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 26256
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 16454
    })
})

In [12]:
# For this task, we aim for the cased base BERT tokenizer 
# Cased because capitalization most likely will play an important role in the NER task

# Import the BERT tokenizer


tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')

In [19]:
# Word Piece tokenization of BERT causes mismatch between actual NER tags and tokens
# As words can be split again by tokenizer (##), as exemplified in the following chunks

example = train_dataset[123]
print(example)

tokenized_example = tokenizer(example['tokens'], is_split_into_words=True)

tokens = tokenizer.convert_ids_to_tokens(tokenized_example['input_ids'])

word_ids = tokenized_example.word_ids()

{'tokens': ['The', 'campaign', 'was', 'organized', ',', 'among', 'others', ',', 'by', 'Abbie', 'Hoffman', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0], 'lang': 'en'}


In [20]:
# Mismatch in length

len(example['ner_tags']), len(tokenized_example['input_ids'])

(12, 16)

In [21]:
# This is due to the presence of special tokens and subwords (typical of BERT), as figured in this example

tokens

['[CLS]',
 'The',
 'campaign',
 'was',
 'organized',
 ',',
 'among',
 'others',
 ',',
 'by',
 'A',
 '##bb',
 '##ie',
 'Hoffman',
 '.',
 '[SEP]']

In [13]:
def tokenize_align(example, label_all_tokens=True):
    '''
    Tokenize and align the NER labels with the corresponding tokens in a given example.

    Args:
        example (dict): A dictionary containing 'tokens' and 'ner_tags' for a single example.
        label_all_tokens (bool, optional): If True, assigns NER labels to all tokens; if False, assigns labels only to the first token of each word.

    Returns:
        dict: Tokenized input with aligned NER labels.

    Note:
        Special tokens are marked with -100, and this will be ignored by PyTorch during training.
        Labels are aligned to their corresponding tokens, considering word boundaries and the label_all_tokens flag.

    '''
    
    tokenized_input = tokenizer(example['tokens'], truncation=True, is_split_into_words=True)
    labels = [] 
    
    for i, label in enumerate(example['ner_tags']):
        word_ids = tokenized_input.word_ids(batch_index=i)
        
        previous_word_idx = None
        label_ids = []
        
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)  # Special tokens will be ignored during training
            
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])  # Assign NER label to the token
            
            else:
                # If word_idx is not a special token,
                # append the NER tag if label_all_tokens is true, else assign -100
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                    
            previous_word_idx = word_idx
            
        labels.append(label_ids)
    
    # Add processed labels to the tokenized input dictionary
    tokenized_input['labels'] = labels
    
    return tokenized_input

# System A

In [24]:
# We apply the logic to entire dataset

tokenized_data = multinerd.map(tokenize_align, batched=True)

  0%|          | 0/106 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [25]:
# Import model from HuggingFace

model_a = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=31)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [23]:
# We use the data collator to pad tokens so all samples are of same length

data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
# Define training parameters

training_args_a = TrainingArguments(
    output_dir="./fine_tune_bert_output_a",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging_steps = 1000,
    report_to="wandb",
    run_name = "RISE_A",
    save_strategy="no"
)

In [28]:
# Create a mapping between indices and tags for readability purposes

label_mapping = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-ANIM",
    8: "I-ANIM",
    9: "B-BIO",
    10: "I-BIO",
    11: "B-CEL",
    12: "I-CEL",
    13: "B-DIS",
    14: "I-DIS",
    15: "B-EVE",
    16: "I-EVE",
    17: "B-FOOD",
    18: "I-FOOD",
    19: "B-INST",
    20: "I-INST",
    21: "B-MEDIA",
    22: "I-MEDIA",
    23: "B-MYTH",
    24: "I-MYTH",
    25: "B-PLANT",
    26: "I-PLANT",
    27: "B-TIME",
    28: "I-TIME",
    29: "B-VEHI",
    30: "I-VEHI",
}

label_names_mapped = [label_mapping[label] for label in label_mapping]
len(label_names_mapped)

31

In [27]:
# seqeval metric from HuggingFace as metrics, which is adequate for chunking tasks such as NER

metric = datasets.load_metric('seqeval')

In [26]:
example = train_dataset[144]
print(example)

labels = [label_names_mapped[i] for i in example['ner_tags']]
labels

{'tokens': ['However', ',', 'the', 'plans', 'were', 'revived', 'in', 'August', '2017', ',', 'with', 'the', 'announcement', 'that', 'Paul', 'Scheer', 'would', 'be', 'writing', 'the', 'series', '.'], 'ner_tags': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0], 'lang': 'en'}


NameError: name 'label_names_mapped' is not defined

In [25]:
# Test the metric on a small sample

metric.compute(predictions=[labels], references=[labels])

NameError: name 'metric' is not defined

In [31]:
def compute_metrics_a(eval_preds):
    
    '''
    Custom function for evaluation metrics of the NER task. It computes accuracy, precision, recall, and F1.
    
    Parameters:
        eval_preds: a tuple with predicted logits and true labels
        
    Returns:
        dictionary with defined metrics
    '''
    
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)

    predictions = [
        [label_mapping[prediction] for (prediction, l) in zip(batch_preds, label) if l != -100] 
        for batch_preds, label in zip(pred_logits, labels)
    ]

    true_labels = [
        [label_mapping[l] for (prediction, l) in zip(batch_preds, label) if l != -100] 
        for batch_preds, label in zip(pred_logits, labels)
    ]

    results = metric.compute(predictions=predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [38]:
# Train the model
# NOTE: we train for 3 epochs because of limited GPU access
# however, it is advised to train for more epochs in a NER task

trainer_a = Trainer(
    model=model_a,
    args=training_args_a,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_a
)

trainer_a.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1000,0.029000,0.089517,0.861586,0.883222,0.872270,0.973968
2000,0.026800,0.089896,0.880721,0.861038,0.870768,0.974299
3000,0.026700,0.086117,0.866047,0.884432,0.875143,0.974454
4000,0.022700,0.087909,0.871454,0.880697,0.876051,0.974842
5000,0.021700,0.095486,0.862138,0.889571,0.875640,0.974606
6000,0.022100,0.089360,0.861895,0.894098,0.877701,0.974890
7000,0.017500,0.109107,0.865016,0.884477,0.874638,0.974406
8000,0.013500,0.108014,0.860411,0.891633,0.875744,0.974283
9000,0.013500,0.109042,0.853810,0.902165,0.877322,0.974230
10000,0.013400,0.107089,0.869591,0.889571,0.879468,0.975369


In [39]:
# Evaluate on test set 

results = trainer_a.predict(tokenized_data["test"])

In [40]:
pred_logits, labels = results.predictions, results.label_ids
pred_logits = np.argmax(pred_logits, axis=2)

# Flatten the predictions and true labels
flat_true_labels = [label for sequence_labels in labels for label in sequence_labels]
flat_pred_labels = [label for sequence_labels in pred_logits for label in sequence_labels]

# Ignore special tokens (-100) during evaluation
valid_indices = [index for index, label in enumerate(flat_true_labels) if label != -100]

# Use the sklearn classification_report function with label_mapping
report = classification_report(
    np.array(flat_true_labels)[valid_indices],
    np.array(flat_pred_labels)[valid_indices],
    target_names=label_names_mapped
)

print(report)

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           O       0.99      0.99      0.99    329987
       B-PER       0.99      0.99      0.99      7354
       I-PER       1.00      1.00      1.00     11509
       B-ORG       0.98      0.98      0.98      5131
       I-ORG       0.99      0.99      0.99      5693
       B-LOC       1.00      0.99      0.99     19489
       I-LOC       0.99      0.99      0.99      7129
      B-ANIM       0.73      0.79      0.76      3351
      I-ANIM       0.70      0.77      0.73      1658
       B-BIO       0.42      0.74      0.54        34
       I-BIO       0.00      0.00      0.00         0
       B-CEL       0.78      0.93      0.85        56
       I-CEL       0.90      0.82      0.86        22
       B-DIS       0.78      0.82      0.80      2014
       I-DIS       0.76      0.78      0.77       917
       B-EVE       0.96      0.96      0.96       451
       I-EVE       0.96      0.98      0.97       667
      B-FOOD       0.68    

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# System B

In [14]:
# Change the original dataset as to retrieve only the entities we need
# while setting all other entities to 0

valid_ner_tags = [1, 2, 3, 4, 5, 6, 7, 8, 13, 14]

# Define a function to modify ner_tags in each sample
def modify_ner_tags(sample):
    # Iterate through ner_tags and replace invalid values with 0
    sample['ner_tags'] = [tag if tag in valid_ner_tags else 0 for tag in sample['ner_tags']]
    return sample

# Apply the modification to the entire dataset
modified_dataset = multinerd.map(modify_ner_tags)

  0%|          | 0/105024 [00:00<?, ?ex/s]

  0%|          | 0/26256 [00:00<?, ?ex/s]

  0%|          | 0/16454 [00:00<?, ?ex/s]

In [20]:
# Apply the logic to entire dataset

tokenized_data_b = modified_dataset.map(tokenize_align, batched=True)

  0%|          | 0/106 [00:00<?, ?ba/s]

  0%|          | 0/27 [00:00<?, ?ba/s]

  0%|          | 0/17 [00:00<?, ?ba/s]

In [15]:
# Create another label mapping but with the needed tags only

new_label_mapping = {
    0: "O",
    1: "B-PER",
    2: "I-PER",
    3: "B-ORG",
    4: "I-ORG",
    5: "B-LOC",
    6: "I-LOC",
    7: "B-ANIM",
    8: "I-ANIM",
    13: "B-DIS",
    14: "I-DIS",
}

In [16]:
# Redefine metric custom function with new dictionary

def compute_metrics_b(eval_preds):
    pred_logits, labels = eval_preds
    pred_logits = np.argmax(pred_logits, axis=2)

    predictions = [
        [new_label_mapping[prediction] for (prediction, l) in zip(batch_preds, label) if l != -100] 
        for batch_preds, label in zip(pred_logits, labels)
    ]

    true_labels = [
        [new_label_mapping[l] for (prediction, l) in zip(batch_preds, label) if l != -100] 
        for batch_preds, label in zip(pred_logits, labels)
    ]

    results = metric.compute(predictions=predictions, references=true_labels)

    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [21]:
training_args_b = TrainingArguments(
    output_dir="./fine_tune_bert_output_b",
    evaluation_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    #logging_steps = 1000,
    report_to="wandb",
    run_name = "RISE_B",
    save_strategy="no",
)

In [18]:
model_b = AutoModelForTokenClassification.from_pretrained('bert-base-cased', num_labels=15)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
trainer_b = Trainer(
    model=model_b,
    args=training_args_b,
    train_dataset=tokenized_data_b["train"],
    eval_dataset=tokenized_data_b["valid"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_b
)

trainer_b.train()

Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.036300,0.080751,0.792136,0.921323,0.851860,0.972556
1000,0.053000,0.049201,0.894481,0.895352,0.894916,0.982548
1500,0.046300,0.048508,0.877449,0.907296,0.892123,0.981843
2000,0.045300,0.043622,0.913085,0.894397,0.903644,0.983928
2500,0.041600,0.043664,0.903291,0.909972,0.906619,0.984174
3000,0.039900,0.041305,0.908576,0.908194,0.908385,0.984598
3500,0.036400,0.039580,0.901742,0.922852,0.912175,0.985142
4000,0.035500,0.038591,0.919055,0.909360,0.914182,0.985418
4500,0.035300,0.037028,0.927853,0.913029,0.920381,0.986261
5000,0.032500,0.037029,0.917010,0.924247,0.920614,0.986294


TrainOutput(global_step=6564, training_loss=0.03843973269192349, metrics={'train_runtime': 1975.0159, 'train_samples_per_second': 53.176, 'train_steps_per_second': 3.324, 'total_flos': 3033089085287520.0, 'train_loss': 0.03843973269192349, 'epoch': 1.0})

In [29]:
results_b = trainer_b.predict(tokenized_data_b["test"])

In [31]:
pred_logits, labels = results_b.predictions, results_b.label_ids
pred_logits = np.argmax(pred_logits, axis=2)

# Flatten the predictions and true labels
flat_true_labels = [label for sequence_labels in labels for label in sequence_labels]
flat_pred_labels = [label for sequence_labels in pred_logits for label in sequence_labels]

# Ignore special tokens (-100) during evaluation
valid_indices = [index for index, label in enumerate(flat_true_labels) if label != -100]

# Use the sklearn classification_report function with label_mapping
report_b = classification_report(
    np.array(flat_true_labels)[valid_indices],
    np.array(flat_pred_labels)[valid_indices],
    target_names=list(new_label_mapping.values())
)

print(report_b)

              precision    recall  f1-score   support

           O       1.00      0.99      0.99    337859
       B-PER       0.99      0.99      0.99      7354
       I-PER       1.00      1.00      1.00     11509
       B-ORG       0.98      0.98      0.98      5131
       I-ORG       0.99      0.99      0.99      5693
       B-LOC       1.00      0.99      1.00     19489
       I-LOC       0.99      1.00      0.99      7129
      B-ANIM       0.71      0.80      0.75      3351
      I-ANIM       0.67      0.79      0.73      1658
       B-DIS       0.77      0.80      0.78      2014
       I-DIS       0.74      0.76      0.75       917

    accuracy                           0.99    402104
   macro avg       0.89      0.92      0.90    402104
weighted avg       0.99      0.99      0.99    402104

